# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [41]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [42]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [43]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [44]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [45]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [46]:
try:
    session.execute ("CREATE KEYSPACE IF NOT EXISTS udacity_p1 WITH REPLICATION = {'class': 'SimpleStrategy' , 'replication_factor':1}")
except Exception as e:
    print (e)

#### Set Keyspace

In [47]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace ('udacity_p1')


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
#### For table songs_per_session, Since our data is looking for details based on sessionId and itemInSession we will create primary key based on this two columns.

In [48]:
##Query 1:Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

##For table songs_per_session, Since our data is looking for details based on sessionId and itemInSession we will create primary key based on this two columns.

query = "CREATE TABLE IF NOT EXISTS songs_per_session (sessionId int, itemInSession int, artist text, song_title text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print (e)

#### INSERT statements after reading variables from the file

In [49]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_per_session (sessionId,itemInSession, artist, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ##Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [50]:
## SELECT statement to verify the data was entered into the table
query = "select artist,song_title,length from songs_per_session WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
#### For table songs_per_user_session, Since our data is looking for details based on userid and sessionId we will create primary key based on this two columns. We also need to add the column `itemInSessionn` as Clustering Column to sort the data.

In [51]:
query = "CREATE TABLE IF NOT EXISTS songs_per_user_session (userid int, sessionId int, itemInSession int, artist text, song_title text, firstName text, lastName text, PRIMARY KEY ((userid, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print (e)

#### INSERT statements after reading variables from the file

In [52]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements after reading variables from the file
        query = "INSERT INTO songs_per_user_session (userid, sessionId, itemInSession, artist, song_title, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ##Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],  line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [53]:
query = "select artist,song_title,firstName,lastName from songs_per_user_session WHERE userid = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song_title, row.firstname,row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
#### For table songs_userdetail, Since our data is looking for details based on song_title, we will add that as partition key. From there we will need to add another element to make sure the Key is unique. So we will add 'userid' as Composite key. That should be enough to make the row key unique.

In [54]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
##For table songs_userdetail, Since our data is looking for details based on song_title, we will add that as partition key. From there we will need to add another element to make sure the Key is unique. So we will add 'userid' as Composite key. That should be enough to make the row key unique.

query = "CREATE TABLE IF NOT EXISTS songs_userdetail (song_title text, userid int, firstName text, lastName text, PRIMARY KEY (song_title, userid))"

try:
    session.execute(query)
except Exception as e:
    print (e)

#### INSERT statements after reading variables from the file

In [55]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_userdetail (song_title, userid, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [56]:
query = "select firstName,lastName from songs_userdetail WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.firstname,row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [57]:
##Drop the table before closing out the sessions
query = "drop table songs_per_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table songs_per_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table songs_userdetail"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [58]:
session.shutdown()
cluster.shutdown()